#  FoldBy Approach

In this second approach we take a different path, exploiting Dask bags in order to use a foldby operation to perform the needed aggregation.

Apart from the data cleansing, which is exactly equal, this branch of our work is developed hand in hand with the previous one, having in mind the goal of obtaining the same results.

The goal of this notebook is indeed to compare, on the same task, the foldby method against a groupby on a distributed Dask data frame. 

In [2]:
from dask.distributed import Client, SSHCluster
cluster = SSHCluster(
     ["10.67.22.41", "10.67.22.41", "10.67.22.145", "10.67.22.146", '10.67.22.160', '10.67.22.253'],
     worker_options={"nthreads": 1, 'n_workers':4},
     scheduler_options={"dashboard_address": ":9455"}
)
client = Client(cluster)



distributed.deploy.ssh - INFO - distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - Clear task state
distributed.deploy.ssh - INFO - distributed.scheduler - INFO -   Scheduler at:   tcp://10.67.22.41:46517
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.145:33165'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.145:35837'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.145:36981'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.145:43525'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.253:44319'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny

distributed.deploy.ssh - INFO - distributed.worker - INFO -               Threads:                          1
distributed.deploy.ssh - INFO - distributed.worker - INFO -                Memory:                   1.94 GiB
distributed.deploy.ssh - INFO - distributed.worker - INFO -       Local Directory: /root/dask-worker-space/worker-gauydi1o
distributed.deploy.ssh - INFO - distributed.worker - INFO - -------------------------------------------------
distributed.deploy.ssh - INFO - distributed.worker - INFO -       Start worker at:   tcp://10.67.22.146:33977
distributed.deploy.ssh - INFO - distributed.worker - INFO -          Listening to:   tcp://10.67.22.146:33977
distributed.deploy.ssh - INFO - distributed.worker - INFO -          dashboard at:         10.67.22.146:32885
distributed.deploy.ssh - INFO - distributed.worker - INFO - Waiting to connect to:    tcp://10.67.22.41:46517
distributed.deploy.ssh - INFO - distributed.worker - INFO - -----------------------------------------------

distributed.deploy.ssh - INFO - distributed.worker - INFO -       Local Directory: /root/dask-worker-space/worker-531sytdd
distributed.deploy.ssh - INFO - distributed.worker - INFO -                Memory:                   1.94 GiB
distributed.deploy.ssh - INFO - distributed.worker - INFO -       Local Directory: /root/dask-worker-space/worker-dw1r18qi
distributed.deploy.ssh - INFO - distributed.worker - INFO - -------------------------------------------------
distributed.deploy.ssh - INFO - distributed.worker - INFO - -------------------------------------------------
distributed.deploy.ssh - INFO - distributed.worker - INFO -       Start worker at:    tcp://10.67.22.41:42055
distributed.deploy.ssh - INFO - distributed.worker - INFO -          Listening to:    tcp://10.67.22.41:42055
distributed.deploy.ssh - INFO - distributed.worker - INFO -          dashboard at:          10.67.22.41:44353
distributed.deploy.ssh - INFO - distributed.worker - INFO - Waiting to connect to:    tcp://10

In [3]:
client

<Client: 'tcp://10.67.22.41:46517' processes=4 threads=4, memory=7.77 GiB>

In [2]:
import dask
import dask.dataframe as dd
import io
from functools import partial

from fsspec.core import open_files
from tlz import concat

from dask.bag.core import from_delayed
from dask.bytes import read_bytes
from dask.delayed import delayed
from dask.utils import parse_bytes, system_encoding
import pandas as pd
import re
import dask.bag as db
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
from operator import add
from collections import Counter

In [7]:
db_travel = db.read_text('/dataNFS/koln.txt', blocksize='9.455MB')

In [8]:
def clean_round_row(row):
    
    pattern = '[0-9]+\.?[0-9]*'
    findings = re.findall(pattern, row[:-1])
    
    if(len(findings) == len(row[:-1].split(' '))):
        round_findings = [round(float(x),4) for x in findings]
        return round_findings
    else:
        return []

In [9]:
db_cleaned = db_travel.map(clean_round_row).filter(lambda x: len(x)>0)\
                      .map(lambda x: {'time':int(x[0]), 'ID':int(x[1]), 
                                       'x':float(x[2]), 'y':float(x[3]),
                                       'velocity':float(x[4])})

In [10]:
#upper, lower, right, left
upper, lower, right, left = pd.read_csv("/dataNFS/grid_params.csv", header=0)[['upper','lower', 'right','left']].values[0]

# delta_x, delta_y
delta_x, delta_y = pd.read_csv("/dataNFS/grid_params.csv", header=0)[['delta_x','delta_y']].values[0]

In [3]:
df_BS = pd.read_csv("/dataNFS/df_BS.csv", header=0)
df_BS.head()

,x,y,base_ID,horizontal,vertical
0,3769.0203,1826.646344,0,0,0
1,5427.4841,1826.646344,1,1,0
2,7085.9479,1826.646344,2,2,0
3,8744.4117,1826.646344,3,3,0
4,10402.8755,1826.646344,4,4,0


In [4]:
df_grid = pd.read_csv("/dataNFS/df_grid.csv", header=0)
df_grid.head()

,x,y
0,2939.7884,847.189
1,4598.2522,847.189
2,6256.7160,847.189
3,7915.1798,847.189
4,9573.6436,847.189


### Foldby

In [14]:
def hor_ver_func(row):
    
    row['horizontal'] = int((row['x'] - left)//delta_x)
    row['vertical'] = int((row['y'] - lower)//delta_y)
    
    return row

In [15]:
db_HV = db_cleaned.map(hor_ver_func)

In [17]:
def square_match_db(row):
    
    base_targets = df_BS[df_BS['horizontal'] == row['horizontal']]
    base_targets = base_targets[base_targets['vertical'] == row['vertical']]
    base_targets['base_ID'] = base_targets.index
    
    
    
    
    # TO LOOK AT
    if len(base_targets) == 1:
        row['base_ID'] = base_targets['base_ID'].values[0]
        
    
    else: 
        row['base_ID'] = -1
        
        
    return row

In [18]:
db_HV_matched = db_HV.map(square_match_db).filter(lambda x: x['base_ID'] >= 0)

In [19]:
result_fold_db_minmax = db_HV_matched.foldby( key = ['base_ID', 'ID'],
                                              binop = lambda tot, x: {'min':min(tot['min'],x['time']),
                                                                      'max':max(tot['max'],x['time'])},
                                              initial = {'min':np.inf, 'max':-np.inf},
                                              combine = lambda tot1, tot2: {'min':min(tot1['min'],tot2['min']),
                                                                            'max':max(tot1['max'],tot2['max'])},
                                              combine_initial = {'min':np.inf, 'max':-np.inf})

In [20]:
diff_time = result_fold_db_minmax.map(lambda x: {'base_ID':x[0][0],
                                                 'ID':x[0][1],
                                                 'min':x[1]['min'],
                                                 'max':x[1]['max']})\
                                  .map(lambda x: {'base_ID':x['base_ID'],
                                                 'ID':x['ID'],
                                                 'diff_time':x['max']-x['min']})

<img src="images/diff_time_foldby.png" width=300>

In [21]:
mean_per_base = diff_time.foldby(key = 'base_ID',
                                 binop = lambda tot, x: {'sum':tot['sum']+x['diff_time'],
                                                         'n':tot['n']+1},
                                 initial = {'sum':0, 'n':0},
                                 combine = lambda tot1, tot2: {'sum':tot1['sum']+tot2['sum'],
                                                               'n':tot1['n']+tot2['n']},
                                 combine_initial = {'sum':0, 'n':0})\
                         .map(lambda x: {'base_ID':x[0],
                                         'mean':x[1]['sum']/x[1]['n']})\
                         .to_dataframe(meta={'base_ID':int, 'mean':float})
                         

<img src="images/mean_per_base_foldby.png" width=500>

The foldby operation does not provide a method to store the result in different partitions, this implementation indeed implies the final data frame to be small enough to fit one single worker's memory.
The visualize graphs above show that a more in-depth analysis is required.

## Benchmark

In the same way as in the groupby notebook we tried different values of the main parameters of our set-up, varying the number of workers (`n_workers`) for each machine of the cluster, the number of threads per worker (`nthreads`) and the block size (`blocksize`) used to read and import the data.

Our aim is to test the performances of the two different DAG optimizations, doing a comparison in order to find the optimal one in terms of computational time.